In [ ]:
! pip install pandas
! pip install openpyxl

import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit


In [ ]:
import pandas as pd

# Define the file paths
files = ['data/CattleData/Cattle.xlsx', 'data/PoultryData/PoultryFull.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    try:
        df = pd.read_excel(file, engine='openpyxl')
        dfs.append(df)
    except Exception as e:
        
        print(f"Error reading {file}: {e}")
barometer_dt_Cattle = dfs[0]
barometer_dt_Poultry = dfs[1]



In [ ]:
barometer_dt_Cattle.head (100)


In [ ]:
barometer_dt_Poultry 


### Step 02: Create an RDF graph and namespaces.

In [69]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

ontology_file = "output/RDFoutputPigSampleLab5.rdf"
g.parse(ontology_file, format="xml")

# Define your custom namespace for your ontology's properties
LHO = Namespace("https://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")

ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")

# For Cattle Data

In [ ]:
# Loop through rows in the "barometer_dt_Cattle" dataframe
for index, row in barometer_dt_Cattle.iterrows():
    PathogenCode = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    Result = row["Result"]
    LabReference = row["LabReference"]
    DiagnosticTest = row["DiagnosticTest"]
    FarmIdentification = row["FarmID"]
    Date = row["Floored_date"]
    

    
    # Create a unique URI for each sample based on the row index
    CattleSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1CattleSample_{index}")
    
    # Add sample type assertion
    g.add((CattleSample_uri, RDF.type, LHO.CattleSample))
    description = "An individual representing a sample from a cattle."
    g.add((CattleSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((CattleSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((CattleSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a cattle breed of mixed meat and dairy type."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((CattleSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))


    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        encoded_country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{encoded_country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((CattleSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("The Netherland")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))

    
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        encoded_province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{encoded_province}")
        g.add((CattleSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((CattleSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))


           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
    # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((CattleSample_uri, LHO.hasSampleResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if Result == "1.0":
        description = "An individual representing a Positive test result"
    elif Result == "0.0":
        description = "An individual representing a negative test result"
    elif Result == "missing":
        description = "n individual representing an unknown or missing test result."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "LabReference" in row and row["LabReference"]:
        LabReference = row["LabReference"]
    # Create a unique URI for the LabReference based on the value
        LabReference_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{LabReference}")
        g.add((CattleSample_uri, LHO.hasLabReference, LabReference_uri))
        g.add((LabReference_uri, RDF.type, LHO.LabReference))

    if "Floored_date" in row and not pd.isnull(row["Floored_date"]):  # Check for NaN or None values
        Date = str(row["Floored_date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((CattleSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    
    # Add any additional properties related to Date as needed


# Serialize the RDF graph to a file
rdf_output_file = "output/RDFoutputPoultrySampleLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it
turtle_data = g.serialize(format="turtle")
print(turtle_data)


# For Poultry Data

In [ ]:


for index, row in barometer_dt_Poultry.iterrows():
    Quarter = row['Quarter']
    IB = row['IB']
    AnimalType = row['AnimalType']
    Province = row['Province']

    # Create a unique URI for each sample based on the row index
    PoultrySample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1PoultrySample_{index}")

    # Add sample type assertion
    g.add((PoultrySample_uri, RDF.type, LHO.PoultrySample))
    description = "An individual representing a sample from Poultry or Chicken."
    g.add((PoultrySample_uri, RDFS.comment, Literal(description, lang="en")))

    if "Quarter" in row and row["Quarter"]:
        Date = row["Quarter"]
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
        g.add((PoultrySample_uri, LHO.hasDate, Date_uri))
        g.add(( Date_uri, RDF.type, LHO.Date))
        description = "An individual representing a particular Date."
        g.add((PoultrySample_uri, RDFS.comment, Literal(description, lang="en")))

    if "IB" in row and row["IB"]:
        Pathogen = row["IB"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((PoultrySample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Poultry Pathogen."
        g.add((PoultrySample_uri, RDFS.comment, Literal(description, lang="en")))

    if "AnimalType" in row and row["AnimalType"]:
        SampleType = row["AnimalType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        g.add((PoultrySample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, decide.SampleType))
        description = "An individual representing SampleType for Poultry."
        g.add((PoultrySample_uri, RDFS.comment, Literal(description, lang="en")))

    if "Province" in row and row["Province"]:
        Province = row["Province"]
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PoultrySample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))

# Serialize the RDF graph to a file
rdf_output_file = "output/RDFoutputPoultrySampleLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it
turtle_data = g.serialize(format="turtle")
print(turtle_data)


# Query for Poultry

In [74]:
query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <https://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?Province), "#") AS ?ProvinceName)
WHERE {
  ?Sample rdf:type LHO:PoultrySample .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasProvince ?Province .
  
}
"""

# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))
df = pd.DataFrame(data, columns=["Sample", "Pathogen", "SampleType",  "Province"])

# Display the dataframe
df.head(20)

,Sample,Pathogen,SampleType,Province
0,Lab1PoultrySample_0,QX,SS,Noord-Brabant
1,Lab1PoultrySample_1,QX,SS,Noord-Brabant
2,Lab1PoultrySample_2,QX,SS,Gelderland
3,Lab1PoultrySample_3,QX,SS,Overijssel
4,Lab1PoultrySample_4,QX,SS,Gelderland
5,Lab1PoultrySample_5,QX,SS,Gelderland
6,Lab1PoultrySample_6,QX,SS,Noord-Brabant
7,Lab1PoultrySample_7,QX,SS,Limburg
8,Lab1PoultrySample_8,793B,SS,Limburg
9,Lab1PoultrySample_9,QX,SS,Gelderland
